<a href="https://colab.research.google.com/github/dawads/bibliometri/blob/master/network_data_from_diva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
url='http://liu.diva-portal.org/smash/export.jsf?format=csvall2&addFilename=true&aq=[[]]&aqe=[]&aq2=[[{"organisationId":"276","organisationId-Xtra":true},{"publicationTypeCode":["review","article"]},{"publicationStatus":"published"},{"contentTypeCode":["refereed"]}]]&onlyFullText=false&noOfRows=5000&sortOrder=title_sort_asc&sortOrder2=title_sort_asc'

divaData=pd.read_csv(url)


URLError: ignored

In [0]:
headers=list(divaData.columns.values)
#print(headers)

divaDataTwo=divaData[['PID','Categories']]
divaDataTwo['Categories']=divaData['Categories'].str.split(';',-1,False)
#print(divaDataTwo)

subjects=divaDataTwo.set_index(['PID'])['Categories'].apply(pd.Series).stack()

subjects=subjects.reset_index()
#print(subjects)
subjects.columns=['PID','Order','Categories']
subjects.head()

#test=subjects.unique()
#print(allSubjects)
#test=allSubjects.unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PID,Order,Categories
0,266537,0,Teknik och teknologier (2)
1,242842,0,Teknik och teknologier (2)
2,634428,0,Acceleratorfysik och instrumentering (10306)
3,634430,0,Acceleratorfysik och instrumentering (10306)
4,1236429,0,Annan materialteknik (20599)
5,865139,0,Kemi (104)
6,865139,1,Cellbiologi (10604)
7,245484,0,Medicin och hälsovetenskap (3)


In [0]:
nodes = subjects.groupby('Categories').agg({'Categories': np.size}, {'Categories': np.sqrt})
nodes.head()
#vill inte mata ut kvadratroten
#hur får jag aggregaten att bli egna namngivna kolumner?

,Categories
Categories,
Acceleratorfysik och instrumentering (10306),5
Algebra och logik (10103),2
Allmänmedicin (30224),15
Analytisk kemi (10401),17
Anestesi och intensivvård (30201),9


nodes AS ( SELECT row_number() OVER (ORDER BY Categories) AS id, Categories AS label, count() AS p FROM subjects GROUP BY Categories HAVING count() > 0 ),

In [0]:
#subjects.groupby('Categories').size()
#Vill inte funka...


In [0]:
(nodes.assign(row_id=subjects.sort_values(['Categories'], ascending=False)
              .rank(method='first', ascending=False))
              #.cumcount() + 1)
)

ValueError: ignored

https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html